# <center> Где бурить новую скважину </center>
___
___

## <center> Учебный проект по ML </center>
___
___

## <center>Описание проекта</center>

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## <center>Описание данных</center>
   
   - три датасета с данными, описанными ниже;
   - `id` - уникальный идентификатор скважины;   
   - `f0, f1, f2` - три признака точек (неважно, что они означают, но сами признаки значимы); 👌   
   - `product` - объём запасов в скважине (тыс. баррелей).

## <center>Состав работ</center> <a id = 'back'></a>

[1. Загрузка и подготовка данных](#1)   

   - [1.1. Исследование данных](#1.1)   
   - [1.2. Выводы](#1.2)
   
[2. Обучение и проверка модели](#2)   

   - [2.1. Разбиение данных](#2.1)   
   - [2.2. Обучение модели и получение предсказаний](#2.2)   
   - [2.3. Сохранение предсказаний и правильных ответов](#2.3)   
   - [2.4. Средний запас предсказанного сырья и RMSE модели](#2.4)   
   - [2.5. Анализ результатов](#2.5)   
   
[3. Подготовка к расчету прибыли](#3)   

   - [3.1. Подготовка переменных для расчета](#3.1)   
   - [3.2. Рассчет достаточного объема сырья](#3.2)   
   - [3.3. Выводы](#3.3)   

[4. Функция для расчета прибыли](#4)   
   
[5. Расчет рисков](#5)   
   
   - [5.1. Bootstrap с 1000 выборок, распределение прибыли](#5.1)   
   - [5.2. Средняя прибыль, 95%-й доверительный интервал](#5.2)   
   - [5.3. Выводы](#5.3)

# <center>Ход работы</center>

Импорт необходимых библиотек.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

<a id = '1'></a>
# 1. Загрузка и подготовка данных
___
___

<a id = '1.1'></a>
### 1.1. Исследование данных.
___

Считываем данные.

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Просматриваем общую информацию.

In [3]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропуски в данных не выявлены. Типы данных выглядят логичными. Посмотрим случаные выборки по каждому DataFrame-у.

In [6]:
geo_data_0.sample(10)

,id,f0,f1,f2,product
47187,WiHTX,-0.111438,0.235642,8.788540,109.345745
61192,bEoMV,0.563249,0.790461,1.932245,77.471715
7955,Eo6VY,1.734093,-0.248220,3.955052,162.705319
76797,kFvZM,-0.270678,1.002545,3.585119,66.524655
70043,JWBhl,0.885154,0.105498,-0.921341,63.022351
77608,A1Wxt,0.269374,0.896275,-1.987669,61.269401
37416,xKk3L,0.092062,0.232094,5.909394,89.218910
20819,L6LxI,-0.001630,0.910443,-0.948416,11.864636
15204,H3XMf,0.927125,0.034456,6.705612,155.197633
43747,K9ipM,-0.250953,0.888874,2.125832,129.047059


In [7]:
geo_data_1.sample(10)

,id,f0,f1,f2,product
12999,M4vbm,1.233715,5.065182,1.991269,53.906522
75498,s8DMx,9.113936,1.303648,5.002937,134.766305
91824,xh4N3,-0.239145,-2.745961,1.997401,57.085625
74386,dTl2E,-7.185631,-3.365844,3.994409,110.992147
31442,Bet7f,-10.135265,-12.321988,2.005799,57.085625
23759,j4xun,7.486033,-6.718768,0.002838,0.000000
17241,PogT9,-16.943346,-4.252636,3.004213,84.038886
89618,HZ9AB,-2.762769,-15.254798,2.990569,84.038886
58054,n7Iqj,-7.969276,-17.636815,1.993197,57.085625
86722,HN1yj,2.099051,-7.042455,3.002240,80.859783


In [8]:
geo_data_2.sample(10)

,id,f0,f1,f2,product
25745,7dfwv,-2.304091,0.014520,7.682439,166.130134
92361,ANLfQ,-0.348385,1.751950,5.072449,92.380305
36518,4x6Rg,0.306517,1.923438,1.417978,120.369447
21579,rwXSE,-0.236579,1.857880,2.065170,50.384236
35889,ZTPnD,-1.300160,0.044068,-0.983820,53.024432
88926,aOlbp,0.801970,1.451340,8.568437,136.211753
66409,QtOsC,-1.595949,0.981343,3.126425,51.581960
86535,F6vdg,3.147743,-1.159563,5.927094,37.058733
38702,6luLj,1.837706,-0.840713,3.651428,74.648215
40685,sTvJ1,3.576620,-1.178782,-0.192776,164.393474


Понятно, что ничего не понятно, но данные, как сказано, важные. Критического ничего не выявлено. Однако в одной выборке промелькнул нулевой объем запасов в скважине. Посмотрим статистику по каждому из наборов.

In [9]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [10]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


И снова ничего аномального. Проверим ещё дубликаты.

In [12]:
geo_data_0.duplicated().sum()

0

In [13]:
geo_data_1.duplicated().sum()

0

In [14]:
geo_data_2.duplicated().sum()

0

Дубликатов нет. С предобработкой данных всё понятно - она не требуется. Займемся подготовкой.   
Мы имеем дело с задачей линейной регрессии. Следовательно необходимо:   
1. Избавится от категориального признака `id`;
2. Привести численные признаки к одному масштабу.

In [15]:
geo_data_0['id'].nunique()

99990

In [16]:
geo_data_1['id'].nunique()

99996

In [17]:
geo_data_2['id'].nunique()

99996

<a id = 'reason'></a>
Почти все значения `id` в наших наборах данных уникальны. Первый мыслью было применить прямое или порядковое кодирование. Однако для данного случая:   
- минус прямого кодирования: добавлять 100000 столбцов? Глупо. Затратно. Не надо.
- минус порядкового кодирования: да, добавится 1 столбец с числами от 0 до 99999. Но! По факту `id` - не полноценный признак, это просто придуманное название для идентификации скважины. Ничего не поменяется от того, что мы назовем её по другому. Кроме того, этот признак не обеспечивает повторяемости других признаков, как, например, в задаче про аэропорты из теоретической части. Т.е.: если у нас есть авиакомпания, положим, "авиакомпания", то в большинстве случаев плюсы и минусы этой авиакомпании будут примерно одинаковыми. А если у нас есть скважина "скважина", и мы другую так же назовем "скважина" - это не будет значить, что их признаки будут примерно одинаковыми.   

Ввиду вышесказанного предлагается просто удалить столбцы `id`. В дальнейшем можно будет реализовать восстановление идентификации конкретной скважины по ее признакам (как выяснилось выше - из 3-х наборов данных, по 100000 вхождений, нет ни одной дублированной записи)

In [18]:
geo_data_0.drop(columns = ['id'], inplace = True)
geo_data_1.drop(columns = ['id'], inplace = True)
geo_data_2.drop(columns = ['id'], inplace = True)

Проверим удаление колонок.

In [19]:
geo_data_0.columns

Index(['f0', 'f1', 'f2', 'product'], dtype='object')

In [20]:
geo_data_1.columns

Index(['f0', 'f1', 'f2', 'product'], dtype='object')

In [21]:
geo_data_2.columns

Index(['f0', 'f1', 'f2', 'product'], dtype='object')

Всё нормально, заодно увидели что имена колонок не требуют корректировок. Необходимо еще отмасштабировать признаки, но этим займемся после разбиения данных [тут](#2.1).

P.s.: появилась идея объединить все данные в один датафрейм и обучить отдельную модель на нём тоже. Создадим DataFrame.

In [22]:
geo_data_full = pd.concat([geo_data_0, geo_data_1, geo_data_2])
geo_data_full.shape

(300000, 4)

In [23]:
geo_data_full.sample(10)

,f0,f1,f2,product
56202,0.624318,0.618280,3.625438,101.250037
56662,13.009005,-9.493345,3.996087,107.813044
20634,0.007031,1.845669,1.895257,24.271354
76201,-10.350675,-6.427865,0.999644,30.132364
22211,0.976977,2.095971,0.943385,64.278045
65947,0.672837,-0.408983,11.221155,164.131032
65965,0.939506,0.125496,1.831852,131.724170
12071,0.657794,-0.398884,1.579647,132.058446
74217,0.542279,0.841057,3.325311,150.570384
62140,12.864829,-0.834672,0.989542,26.953261


Объединение прошло. Посмотрим, что из этого получится.

___
[*К содержанию*](#back)

<a id = '1.2'></a>
### 1.2. Выводы.
___

В полученных данных:   
   - отсутствуют пропуски;   
   - отсутствуют дубликаты;
   - типы данных логичны;   
   - аномалий в распределении значений не обнаружено;   
   - предобработка не требуется;   
   - была удалена колонка `id`, как ненужная (обоснавание читать [тут](#reason) )
   - масштабирование оставили для пункта 2.1.

___
[*К содержанию*](#back)

<a id = '2'></a>
# 2. Обучение и проверка модели
___
___

<a id = '2.1'></a>
### 2.1. Разбиение данных.
___

Сформируем целевые и характерные признаки для каждого набора данных.

In [24]:
target_0 = geo_data_0['product']
features_0 = geo_data_0.drop(['product'], axis = 1)
target_1 = geo_data_1['product']
features_1 = geo_data_1.drop(['product'], axis = 1)
target_2 = geo_data_2['product']
features_2 = geo_data_2.drop(['product'], axis = 1)

In [25]:
target_full = geo_data_full['product']
features_full = geo_data_full.drop(['product'], axis = 1)

Разобьем данные на обучающие и валидационные выборки.

In [26]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size = 0.25, random_state = 481516)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size = 0.25, random_state = 481516)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size = 0.25, random_state = 481516)

In [27]:
features_train_full, features_valid_full, target_train_full, target_valid_full = train_test_split(
    features_full, target_full, test_size = 0.25, random_state = 481516)

Отмасштабируем характерные признаки для каждой из выборок.

In [28]:
scaler = StandardScaler()
scaler.fit(features_train_0)
features_train_0 = scaler.transform(features_train_0)
features_valid_0 = scaler.transform(features_valid_0)

In [29]:
scaler.fit(features_train_1)
features_train_1 = scaler.transform(features_train_1)
features_valid_1 = scaler.transform(features_valid_1)

In [30]:
scaler.fit(features_train_2)
features_train_2 = scaler.transform(features_train_2)
features_valid_2 = scaler.transform(features_valid_2)

In [31]:
scaler.fit(features_train_full)
features_train_full = scaler.transform(features_train_full)
features_valid_full = scaler.transform(features_valid_full)

Разбили и отмасштабировали, едем дальше.

___
[*К содержанию*](#back)

<a id = '2.2'></a>
### 2.2. Обучение модели и получение предсказаний.
___

Модель для нулевого региона (вообще логичнее - первого... Но чтобы в цифрах не путаться, пусть будет нулевой).

In [32]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0)

Модель для первого региона.

In [33]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)

Модель для второго региона.

In [34]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)

Модель для объединённых по регионам данных.

In [35]:
model_full = LinearRegression()
model_full.fit(features_train_full, target_train_full)
predicted_valid_full = model_full.predict(features_valid_full)

___
[*К содержанию*](#back)

<a id = '2.3'></a>
### 2.3. Сохраняем предсказания и правильные ответы.
___

In [36]:
predictions = [predicted_valid_0, predicted_valid_1, predicted_valid_2, predicted_valid_full]
answers = [target_valid_0, target_valid_1, target_valid_2, target_valid_full]

___
[*К содержанию*](#back)

<a id = '2.4'></a>
### 2.4. Средний запас предсказанного сырья и RMSE модели.
___

In [37]:
for i in range(len(predictions)):
    print('Средний запас сырья, предсказанный для одной точки в регионе', i, 'равен {:.2f}'.format(predictions[i].mean()))

Средний запас сырья, предсказанный для одной точки в регионе 0 равен 92.53
Средний запас сырья, предсказанный для одной точки в регионе 1 равен 68.89
Средний запас сырья, предсказанный для одной точки в регионе 2 равен 94.85
Средний запас сырья, предсказанный для одной точки в регионе 3 равен 85.54


In [38]:
for i in range(len(predictions)):
    print('Запас сырья, предсказанный для региона', i, 'равен {:.2f}'.format(predictions[i].sum()))

Запас сырья, предсказанный для региона 0 равен 2313273.61
Запас сырья, предсказанный для региона 1 равен 1722339.06
Запас сырья, предсказанный для региона 2 равен 2371127.55
Запас сырья, предсказанный для региона 3 равен 6415223.69


In [39]:
for i in range(len(predictions)):
    print('Модель {}, MSE {:.4f}, RMSE {:.4f}'.format(i, mean_squared_error(answers[i], predictions[i]), mean_squared_error(answers[i], predictions[i]) ** 0.5))

Модель 0, MSE 1429.7733, RMSE 37.8123
Модель 1, MSE 0.7886, RMSE 0.8880
Модель 2, MSE 1606.6521, RMSE 40.0831
Модель 3, MSE 1505.4619, RMSE 38.8003


Примечание: в данных выводах под моделью 3 имеется в виду модель, обученная по объединенным данным.

___
[*К содержанию*](#back)

<a id = '2.5'></a>
### 2.5. Анализ результатов.
___

Результаты для моделей 0, 2 и 3 выглядят разумными. Модель 1 показала слишком низкие значения метрик `MSE` и `RMSE`. Проверим, на всякий случай, модели на адекватность.

In [40]:
predicted_valid_mean_list = [
    pd.Series(target_train_0.mean(), index = target_valid_0.index),
    pd.Series(target_train_1.mean(), index = target_valid_1.index),
    pd.Series(target_train_2.mean(), index = target_valid_2.index),
    pd.Series(target_train_full.mean(), index = target_valid_full.index)
]

In [41]:
for i in range(len(answers)):
    print('Модель {}, MSE {:.4f}, RMSE {:.4f}'.format(i, mean_squared_error(answers[i], predicted_valid_mean_list[i]), mean_squared_error(answers[i], predicted_valid_mean_list[i]) ** 0.5))

Модель 0, MSE 1977.3623, RMSE 44.4675
Модель 1, MSE 2109.2036, RMSE 45.9261
Модель 2, MSE 2000.9307, RMSE 44.7318
Модель 3, MSE 2161.6001, RMSE 46.4930


Итог: с моделью 1 явно что-то не так. Модель 0 неплоха, но модель 2 показала себя наилучшим образом, с разницей в RMSE всего примерно на 4 единицы. С этой моделью далее и будем работать.

Дополнение: модель 3, обученная по объединенным данным, тоже показала себя неплохо: немного лучше чем модель 1, но, всё же, хуже, чем модель 2. Предыдущее утверждение верно: продолжаем работать со второй моделью.

Дополнение 2 (больше для себя): задался вдруг вопросом "а если обучить на отмасштабированных признаках, а обучить на неотмасштабированных?". Логикой понимал что, ммм, нелогичная затея, даже причины придумал. Но попробавать интересно. Итог: было закоменчено масштабирование валидационной выборки, предсказания получились очень хорошими, но RMSE по константным данным меньше почти в два раза, чем у валидных данных. Собственно - плохая идея)

___
[*К содержанию*](#back)

<a id = '3'></a>
# 3. Подготовка к расчёту прибыли
___
___

<a id = '3.1'></a>
### 3.1. Подготовка переменных для расчета.
___

Создаем константы на основании предоставленных нам данных.

In [42]:
BUDGET = 10000000000       #бюджет на разработку скважин, руб.
RESEARCHED_BORE = 500      #количество точек исследования в регионе, шт.
ML_BORE = 200              #количество точек для модели ML, шт. (из 500 точек)
ONE_BARREL_PRICE = 450     #доход с одной баррели сырья, руб.
ONE_THOUSAND_BARREL_PRICE = 450000       #доход с 1000 баррелей сырья, руб.
ACCEPTABLE_PROBABILITY_OF_LOSS = 0.025   #допустимая вероятность убытков. (2,5%)

___
[*К содержанию*](#back)

<a id = '3.2'></a>
### 3.2. Рассчет достаточного объема сырья.
___

У нас нет информации о расходах, только допустимый бюджет на разработку.   
У нас нет информации о финансовых обязательствах компании, операционных расходах и т.д., поэтому будем считать, что 450 рублей это чистая прибыль компании с продажи одной баррели нефти.   
У нас нет информации, какую итоговую прибыль хочет получить компания с продажи сырья.   
Значит за достаточный объем сырья будем принимать тот объем, продажи которого как минимум должны покрыть бюджет на разработку скважин, а потом уже будем выбирать те регионы, которые впоследствии принесут и больший доход.   
Тогда:

In [43]:
sufficient_volume = BUDGET / ONE_BARREL_PRICE
print('Достаточный объем сырья с двухсот точек: {:.2f} баррелей или {:.2f} тыс. баррелей (в размерности предоставленных данных)'.format(sufficient_volume, sufficient_volume/1000))

Достаточный объем сырья с двухсот точек: 22222222.22 баррелей или 22222.22 тыс. баррелей (в размерности предоставленных данных)


In [44]:
print('Достаточный объем сырья с одной точки: {:.2f} баррелей или {:.2f} тыс. баррелей (в размерности предоставленных данных)'.format(sufficient_volume / 200, sufficient_volume/200000))

Достаточный объем сырья с одной точки: 111111.11 баррелей или 111.11 тыс. баррелей (в размерности предоставленных данных)


Для удобства далее 1 тыс. баррелей будем называть 1 еденицей продукта (ед. прод.) как указано в ТЗ, и в каком виде нам предоставлены данные.

___
[*К содержанию*](#back)

<a id = '3.3'></a>
### 3.3. Выводы.
___

Что ж, ранее лучшая модель нам предсказала на одну точку среднее значение, равное 94.85 ед. прод.   
Не самый лучший результат, но, возможно, что это связанно с представленными данными, в частности с тем, что количество выборок аж 100000, что, разумеется, благотворно сказывается на обучении моделей. При этом при статистическом анализе предоставленых данных выявлено, что в каждом наборе максимальный объем ед. прод выше требуемого, а в двух наборах данных даже 75%-й квантиль выше 111,11 ед. прод.   
В любом случае дальше это будет проверено ещё раз.

___
[*К содержанию*](#back)

<a id = '4'></a>
# 4. Функция для расчета прибыли.
___
___

Напишем функцию для расчета прибыли.   
Напомним: в регионе исследуются 500 точек, из которых выбираются 200 с наилучшими предсказаниями.   
Логика работы функции:
   - в качестве аргумента функция принимает ответы, предсказания и количество скважин с наилучшими показателями;
   - полученные предсказания сортируются по убыванию;
   - получаем индексы лучших предсказаний;
   - получаем реальные данные об объеме сырья по индексам;
   - суммируются первые 200 реальных значений объема сырья;
   - полученное значение умножается на стоимость ед. прод. и возвращается как прибыль.

In [45]:
def profit_calculation(target, predictions, count = ML_BORE):
    
    predictions = pd.Series(predictions)
    predictions.index = target.index
    
    pred_sorted = predictions.sort_values(ascending = False)
    selected = target[pred_sorted.index][:ML_BORE]

    total_profit = (selected.sum()*ONE_THOUSAND_BARREL_PRICE) - BUDGET
            
    return total_profit

Проверим работоспособность функции на полученных ранее предсказаниях.

In [46]:
for i in range(len(answers)):
    print('Прибыль с региона ', i, 'равна', profit_calculation(answers[i], predictions[i], ML_BORE))

Прибыль с региона  0 равна 3194884841.8524227
Прибыль с региона  1 равна 2415086696.681511
Прибыль с региона  2 равна 2577222625.788872
Прибыль с региона  3 равна 786445858.3951969


Хорошая прибыль, покрывает расходы. Однако помним что выборка была 100000, и при исследовании всего 500 точек, из которых будут браться 200 для ML, не факт что получим такие же значения.

___
[*К содержанию*](#back)

<a id = '5'></a>
# 5. Расчёт рисков 
___
___

<a id = '5.1'></a>
### 5.1. Bootstrap с 1000 выборок, распределение прибыли.
___

Для применения техники Bootstrap удобнее будет написать функцию.

In [47]:
state = np.random.RandomState(481516)

def bootstrap_1000(target, predictions, count = ML_BORE):
    
    predictions = pd.Series(predictions)
    profit_list = []
    predictions.index = target.index
    
    for i in range(count):
        target_subsample = target.sample(RESEARCHED_BORE, replace = True, random_state = state)
        pred_subsample = predictions[target_subsample.index]
        
        profit = profit_calculation(target_subsample, pred_subsample)
        profit_list.append(profit)
    
    profit_list = pd.Series(profit_list)
    
    return profit_list

Составим список прибылей по регионам (в этот раз объединённые данные не учитываем).

In [48]:
regions_profit_list = []
for i in range(3):
    regions_profit_list.append(bootstrap_1000(answers[i], predictions[i]))

In [49]:
for i in range(len(regions_profit_list)):
    print('Распределение прибыли по региону', i, 'от', regions_profit_list[i].min(), 'до', regions_profit_list[i].max())

Распределение прибыли по региону 0 от -157752128.57635117 до 1396619167.0686264
Распределение прибыли по региону 1 от -78815613.45598984 до 1047747764.5515041
Распределение прибыли по региону 2 от -739283794.9509277 до 925734190.4122505


___
[*К содержанию*](#back)

<a id = '5.2'></a>
### 5.2. Средняя прибыль, 95%-й доверительный интервал.
___

Напишем функцию поиска средней прибыли и 95% доверительного интервала.

In [50]:
#def print_stats(probs):
#    for proba in probs:
#        print('Среднее значение прибыли равно: ', proba.mean())
#        print('95% доверительный интервал равен:', st.t.interval(0.95, len(proba) - 1, loc = proba.mean(), scale = proba.sem()))

In [51]:
def print_stats(probs):
    for proba in probs:
        print('Среднее значение прибыли равно: ', proba.mean())
        print('95% доверительный интервал равен:', proba.quantile(0.025), ':', proba.quantile(0.975))

In [52]:
print_stats(regions_profit_list)

Среднее значение прибыли равно:  480964071.09406424
95% доверительный интервал равен: -57588346.97523742 : 1002826516.4026724
Среднее значение прибыли равно:  501897578.7879619
95% доверительный интервал равен: 65229870.14375734 : 907389176.061705
Среднее значение прибыли равно:  418491837.4540251
95% доверительный интервал равен: -157112773.27931565 : 891344354.2704538


Определим риски.
Так как по определению ТЗ риск есть отрицательная прибыль, то идея следующая: для полученных списков прибылей из каждой вычитаем бюджет на разработку скважин, подсчитываем количество отрицательных значений и делим его на длину списка прибылей. Напишем под это функцию.

In [53]:
def risk(profit):
    negative_profit_count = 0
    for i in range(len(profit)):
        if (profit[i]) < 0:
            negative_profit_count += 1
    return negative_profit_count/len(profit) * 100

In [54]:
for i in range(len(regions_profit_list)):
    print('Риск убытка для региона', i, 'равен', risk(regions_profit_list[i]), '%')

Риск убытка для региона 0 равен 4.0 %
Риск убытка для региона 1 равен 1.0 %
Риск убытка для региона 2 равен 7.000000000000001 %


___
[*К содержанию*](#back)

<a id = '5.3'></a>
### 5.3. Выводы.
___

Итак, самым благоприятным для разработки признаётся регион 1:
- минимальный риск убытков;
- наибольшее среднее значение прибыли;
- наиболее благоприятный доверительный интервал для среднего значения прибыли;
- это единственный регион, который в принципе показал вероятность убытков меньше допустимой по ТЗ (1% < 2.5%)

___
[*К содержанию*](#back)